# EquiHealth: Health Data Collection

This notebook will be used to **implement the collection of health-related datasets** for the EquiHealth project. 
The goal is to gather structured data that can later be used to analyze healthcare inequalities across districts and support the hospital complaint portal.

---

## Objectives

- Identify sources for multiple health datasets, including hospital facilities and population statistics.
- Plan the steps required to clean, standardize, and store data for future analysis and visualization.
- Outline the structure of processed datasets for dashboards and reporting.

---

## Planned Target Datasets

1. **Hospital / Healthcare Facility Data**
   - Sources: National Health Portal Hospital Directory, Rural Health Statistics (RHS), HMIS portal
   - Planned fields: Name, Type, Ownership, Address, District, State, Pincode, Contact info, Services, Beds, Geo-coordinates, Last updated, Source URL

2. **Population Data**
   - Sources: Census of India, district/state population projections
   - Planned fields: District/City name and code, Total population, Age groups, Urban/Rural split, Population density, Source URL, Year

3. **Other Health Indicators (Future / Optional)**
   - Staff availability, equipment, sanctioned vs functional beds, program coverage
   - Sources: HMIS, Ministry of Health reports, government portals

---

## Planned Data Fields

- **Facility Table (to be created):**  
  `facility_id`, `name`, `type`, `ownership`, `address`, `district`, `state`, `pincode`, `latitude`, `longitude`, `contact`, `services`, `beds`, `last_updated`, `source_url`

- **Population Table (to be created):**  
  `district_code`, `district_name`, `state_name`, `total_population`, `age_groups`, `urban_rural`, `source_url`, `year`

- **Optional Metrics / Indicators (to be considered later):**  
  Staff counts, ICU beds, equipment availability, program coverage

---

## Planned Workflow

1. Identify official sources and APIs for hospital, population, and other health datasets.
2. Plan how to download or scrape raw datasets.
3. Plan data cleaning and normalization steps (facility names, addresses, district codes).
4. Plan merging and transformation into master tables for analysis.
5. Plan saving processed data in `data/processed/` for visualization and portal integration.


## Scraping Hospital Data from Jeevandayee Portal (Maharashtra)

The [**Jeevandayee Portal**](https://www.jeevandayee.gov.in/MJPJAY/FrontServlet?requestType=PublicViewsRH&actionVal=ViewBedInfoForDisease&City=x%20&Disease=-1&DataFlag=true&DfltHospList=Reports) is the official website for the **Mahatma Jyotiba Phule Jan Arogya Yojana (MJPJAY)**.  
It contains information about empaneled hospitals in **all districts of Maharashtra**.

### Why This Data?
For the **EquiHealth project**, we are collecting hospital data for the whole of India.  
From this portal, we will get hospital data for Maharashtra, including:

- Hospital name  
- District  
- Taluka (sub-district)  
- Address  
- Pincode  
- MCO contact number  
- Total beds  
- Hospital type (Government / Corporate)  
- Government sub-type (for government hospitals)

This data will later be combined with other states’ data to create a **national hospital dataset**.

### Planned Scraping Method
The portal is **interactive**:

- Hospital lists appear after selecting a district and clicking search.  
- Additional details are in separate tables or links.  
- No direct download or API is available.  

We plan to use **Selenium WebDriver** to:

1. Open the portal in a browser.  
2. Scrape **basic hospital info** for all districts.  
3. Enrich data with **detailed hospital info**, including taluka, address, pincode, and MCO contact.  
4. Identify **hospital type** and **Government sub-type**.  
5. Scrape all **hospital service categories**.  
6. Build the **hospital-category mapping table**.

### Output
The planned datasets:

1. `hospital.csv` → Detailed hospital information.  
2. `category.csv` → Hospital service categories.  
3. `hospital_category.csv` → Maps hospitals to categories.

These datasets will form the foundation for **EquiHealth’s analysis and visualization** of healthcare inequalities in Maharashtra.


## Setup: Import Libraries for Selenium and Data Handling

We will use **Selenium** to scrape hospital data from the Jeevandayee portal and **pandas** to store the data in a CSV file.  

**Libraries used:**
- `selenium` – to control the web browser and interact with the portal.
- `pandas` – to store and manipulate the scraped data.
- `os` – to handle file paths.


In [14]:
# Import Selenium modules for browser automation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

# Import pandas for data handling
import pandas as pd

# Import os for file path operations
import os

import time

import requests
from bs4 import BeautifulSoup

import re

import random
from tqdm import tqdm
import numpy as np

## Initialize WebDriver and Wait

We set up the Chrome WebDriver to control the browser and interact with the portal.  

- `driver = webdriver.Chrome()` → Opens a Chrome browser window for automation.  
- `wait = WebDriverWait(driver, 25)` → Waits up to 25 seconds for elements to load before performing actions.  


In [172]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 25)

## Set Portal URL

We store the Jeevandayee Portal URL in a variable.  
This URL points to the **hospital information page** for Maharashtra.

- `hospital_info_url` → The link to the page where hospital data will be scraped.


In [18]:
hospital_info_url = "https://www.jeevandayee.gov.in/MJPJAY/FrontServlet?requestType=PublicViewsRH&actionVal=ViewBedInfoForDisease&City=x%20&Disease=-1&DataFlag=true&DfltHospList=Reports"

## Scrape Hospital Data from the Portal

Steps to get hospital data from the Jeevandayee portal:

1. Open the hospital info page in the browser.  
2. Click the **Get Information** button to load the table.  
3. Wait for the hospital table to appear.  
4. Get all table rows except the header and footer.  
5. Loop through each row and extract:
   - Hospital Name  
   - District  
   - Total Number of Beds  
6. Save the extracted data in a list of dictionaries (`hospital_data_list`) for later use.


In [ ]:
driver.get(hospital_info_url)  # Open the hospital info page

get_info_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Get Information']")))
get_info_button.click()  # Click the "Get Information" button

hospital_table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[@class='tabBorder']")))  # Wait for table
table_rows = hospital_table.find_elements(By.XPATH, ".//tr[position()>1 and position()<last()]")  # Skip header/footer

hospital_data_list = []  # Store scraped data

for row in table_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) >= 4:
        hospital_name = cells[1].text.strip()
        district = cells[2].text.strip()
        total_beds = cells[3].text.strip()
        
        hospital_data_list.append({
            "Hospital Name": hospital_name,
            "District": district,
            "Total Number of Beds": total_beds
        })

In [92]:
hospital_df = pd.DataFrame(hospital_data_list)
hospital_df

,Hospital Name,District,Total Number of Beds
0,INDRAVATI HOSPITAL & RESEARCH CENTRE,THANE,30
1,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,THANE,100
2,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,MUMBAI,15
3,SANJEEVANI HOSPITAL,NAGPUR,30
4,A & G HOSPITAL,THANE,50
...,...,...,...
2369,suryodaya hospital,NASHIK,50
2370,sushrut Hospital,DHARASHIV,30
2371,suvidha hospital and icu centre,DHARASHIV,50
2372,swara hospital,JALGAON,30


## Count Unique Hospital-District Combinations

Check how many **unique hospital and district pairs** are in the DataFrame.  
This helps verify if there are duplicate entries in the scraped data.


In [93]:
len(hospital_df[["Hospital Name", "District"]].drop_duplicates())

2370

## Remove Duplicate Hospital Entries

Remove duplicate rows based on **Hospital Name and District**.  
Keep the first occurrence and reset the index for a clean DataFrame.


In [94]:
hospital_df = hospital_df.drop_duplicates(subset=["Hospital Name", "District"], keep="first").reset_index(drop=True)

In [98]:
hospital_df

,Hospital Name,District,Total Number of Beds
0,INDRAVATI HOSPITAL & RESEARCH CENTRE,THANE,30
1,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,THANE,100
2,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,MUMBAI,15
3,SANJEEVANI HOSPITAL,NAGPUR,30
4,A & G HOSPITAL,THANE,50
...,...,...,...
2365,suryodaya hospital,NASHIK,50
2366,sushrut Hospital,DHARASHIV,30
2367,suvidha hospital and icu centre,DHARASHIV,50
2368,swara hospital,JALGAON,30


## Update Hospital Details with Additional Info

We now enrich the hospital DataFrame (`hospital_details_df`) with additional columns:  
- HOSP DISP CODE  
- Taluka  
- Address  
- Pincode  
- MCO Contact Number  

Steps:  
1. Navigate to the **Network Hospitals** section on the portal.  
2. Click **All** to view all hospitals.  
3. Scrape detailed information from the table.  
4. Match hospitals by name and district, and update the DataFrame.


In [ ]:
# Create a copy of the DataFrame and add new columns
hospital_details_df = hospital_df.copy()
hospital_details_df["HOSP DISP CODE"] = None
hospital_details_df["Taluka"] = None
hospital_details_df["Address"] = None
hospital_details_df["Pincode"] = None
hospital_details_df["MCO Contact Number"] = None

# Open the portal
driver.get(hospital_info_url)

# Navigate to Network Hospitals menu
network_menu = wait.until(EC.presence_of_element_located((By.LINK_TEXT, "Network Hospitals")))
webdriver.ActionChains(driver).move_to_element(network_menu).perform()
network_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@onclick='javascript:viewHosp()']")))
network_link.click()

# Click "All" link to view all hospitals
all_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@class='tableBluelnks' and text()='All']")))
driver.execute_script("arguments[0].scrollIntoView();", all_link)
driver.execute_script("arguments[0].click();", all_link)

# Wait for hospital table and get rows
hospital_table = wait.until(EC.presence_of_element_located((By.ID, "myTable")))
table_rows = hospital_table.find_elements(By.XPATH, ".//tr[position()>1 and position()<last()]")

updated = 0  # Counter for updated hospitals
for row in table_rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) >= 10:
        hosp_name = cells[5].text.strip()
        district_name = cells[3].text.strip()
        hosp_disp_code = cells[2].text.strip() or None
        taluka = cells[4].text.strip() or None

        # Extract address from link if available
        try:
            address_link = cells[6].find_element(By.TAG_NAME, "a")
            address = address_link.get_attribute("href") or address_link.get_attribute("onclick")
            address = address.split("viewAddress('")[1].split("',")[0].strip()
        except:
            address = cells[6].text.strip() or None

        # Get pincode if available
        try:
            pincode = cells[7].text.strip() or None
        except IndexError:
            pincode = None

        # Get MCO contact number if available
        try:
            mco_contact = cells[8].text.strip() or None
        except IndexError:
            mco_contact = None

        # Match hospital by name and district, then update DataFrame
        mask = (
            (hospital_details_df["Hospital Name"].str.strip() == hosp_name) &
            (hospital_details_df["District"].str.strip() == district_name)
        )
        if mask.any():
            hospital_details_df.loc[mask, "HOSP DISP CODE"] = hosp_disp_code
            hospital_details_df.loc[mask, "Taluka"] = taluka
            hospital_details_df.loc[mask, "Address"] = address
            hospital_details_df.loc[mask, "Pincode"] = pincode
            hospital_details_df.loc[mask, "MCO Contact Number"] = mco_contact
            updated += 1

# Print total number of hospitals updated
print(f"Hospitals updated: {updated}")

Hospitals updated: 2101


In [162]:
hospital_details_df

,Hospital Name,District,Total Number of Beds,HOSP DISP CODE,Taluka,Address,Pincode,MCO Contact Number
0,INDRAVATI HOSPITAL & RESEARCH CENTRE,THANE,30,IAAHC,NA,"Plot no. GHANSOLI VILLAGE, RCC HOUSE, ARJUNE W...",400701,8452862559
1,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,THANE,100,MPCT,NA,"C7, Budhyadev Mandir Marg, Sector 4, Sanpada, ...",400705,9702693695
2,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,MUMBAI,15,None,None,None,None,None
3,SANJEEVANI HOSPITAL,NAGPUR,30,SANEI,NA,"NEAR TAHSIL OFFICE, GOTMARE LAYOUT, BRAHMI KA...",441501,9860447504
4,A & G HOSPITAL,THANE,50,AGAHT,NA,Suchak House murbad road opp karnik road kalya...,421301,9326114930
...,...,...,...,...,...,...,...,...
2365,suryodaya hospital,NASHIK,50,SURY,Nashik,"2,near dwarka,renuka nagar,near dwarka,NASHIK",422001,7588194242
2366,sushrut Hospital,DHARASHIV,30,SURHH,NA,"Plot No. 27 / 1174, samarth nagar, waruda roa...",413501,9422069885
2367,suvidha hospital and icu centre,DHARASHIV,50,SHI,Osmanabad,"28/438,28/439,Dic Road ,Barshi Bypass,Near Hp ...",413501,9890922151
2368,swara hospital,JALGAON,30,SWAR,NA,"12B,behind brookebond coloney ,ringroad,near b...",425001,9922307744


## Add Hospital Type Information

We now update each hospital with its **type (Government or Corporate)**:

1. Copy the detailed hospital DataFrame.  
2. Navigate to the portal and select each hospital type from the dropdown.  
3. Click "Get Information" to load the hospital table.  
4. Match hospitals by name and district, then update the **Hospital Type** column.  
5. Print how many hospitals were updated for each type.


In [ ]:
# Define hospital types
hospital_types = {"G": "Government", "C": "Corporate"}

# Create a copy of the hospital DataFrame and add a new column for type
hospital_with_type_df = hospital_details_df.copy()
hospital_with_type_df["Hospital Type"] = None

# Open the portal
driver.get(hospital_info_url)

try:
    # Loop through each hospital type
    for type_value, type_name in hospital_types.items():
        try:
            # Locate hospital type dropdown
            hosp_type_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "OptionHosp")))
        
            # Select the option matching the current type
            for option in hosp_type_dropdown.find_elements(By.TAG_NAME, "option"):
                if option.get_attribute("value") == type_value:
                    option.click()
                    break
            
            # Click "Get Information" button
            get_info_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Get Information']")))
            get_info_button.click()
            
            # Wait for hospital table to load
            hospital_table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[@class='tabBorder']")))
            table_rows = hospital_table.find_elements(By.XPATH, ".//tr[position()>1 and position()<last()]")
            
            updated_count = 0
            for row in table_rows:
                cells = row.find_elements(By.TAG_NAME, "td")                    
                if len(cells) >= 3:
                    hosp_name = cells[1].text.strip()
                    district = cells[2].text.strip()
                    # Match hospital by name and district
                    mask = (
                        (hospital_with_type_df["Hospital Name"].str.strip() == hosp_name) &
                        (hospital_with_type_df["District"].str.strip() == district)
                    )
                    if mask.any():
                        hospital_with_type_df.loc[mask, "Hospital Type"] = type_name
                        updated_count += mask.sum()
                        
            print(f"Type '{type_name}': {updated_count} hospitals updated.")
                        
        except:
            print(f"No hospitals found for hospital type: {type_name}. Continuing to next type.")

except:
        print("No hospital type dropdown found.")

Type 'Government': 656 hospitals updated.
Type 'Corporate': 1713 hospitals updated.


In [164]:
hospital_with_type_df

,Hospital Name,District,Total Number of Beds,HOSP DISP CODE,Taluka,Address,Pincode,MCO Contact Number,Hospital Type
0,INDRAVATI HOSPITAL & RESEARCH CENTRE,THANE,30,IAAHC,NA,"Plot no. GHANSOLI VILLAGE, RCC HOUSE, ARJUNE W...",400701,8452862559,Corporate
1,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,THANE,100,MPCT,NA,"C7, Budhyadev Mandir Marg, Sector 4, Sanpada, ...",400705,9702693695,Corporate
2,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,MUMBAI,15,None,None,None,None,None,Corporate
3,SANJEEVANI HOSPITAL,NAGPUR,30,SANEI,NA,"NEAR TAHSIL OFFICE, GOTMARE LAYOUT, BRAHMI KA...",441501,9860447504,Corporate
4,A & G HOSPITAL,THANE,50,AGAHT,NA,Suchak House murbad road opp karnik road kalya...,421301,9326114930,Corporate
...,...,...,...,...,...,...,...,...,...
2365,suryodaya hospital,NASHIK,50,SURY,Nashik,"2,near dwarka,renuka nagar,near dwarka,NASHIK",422001,7588194242,Corporate
2366,sushrut Hospital,DHARASHIV,30,SURHH,NA,"Plot No. 27 / 1174, samarth nagar, waruda roa...",413501,9422069885,Corporate
2367,suvidha hospital and icu centre,DHARASHIV,50,SHI,Osmanabad,"28/438,28/439,Dic Road ,Barshi Bypass,Near Hp ...",413501,9890922151,Corporate
2368,swara hospital,JALGAON,30,SWAR,NA,"12B,behind brookebond coloney ,ringroad,near b...",425001,9922307744,Corporate


## Add Government Sub-Type for Hospitals

We now update **Government hospitals** with their specific **sub-type**:

1. Copy the hospital DataFrame that already has types.  
2. Select "Government" in the hospital type dropdown.  
3. Check if a Government sub-type dropdown exists.  
4. Loop through each sub-type and scrape hospital info.  
5. Match hospitals by name, district, and type, then update the **Government Sub-Type** column.  
6. Print how many hospitals were updated for each sub-type.


In [ ]:
# Create a copy of the DataFrame and add a new column for Government Sub-Type
hospital_with_govt_subtype_df = hospital_with_type_df.copy()
hospital_with_govt_subtype_df["Government Sub-Type"] = None

# Open the portal
driver.get(hospital_info_url)

# Select "Government" in hospital type dropdown
hosp_type_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "OptionHosp")))
for option in hosp_type_dropdown.find_elements(By.TAG_NAME, "option"):
    if option.get_attribute("value") == "G":
        option.click()
        break

try:
    # Locate Government sub-type dropdown
    govt_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "OptionGovtHosp")))
    select_govt = Select(govt_dropdown)
    govt_options = [opt.text.strip() for opt in select_govt.options if opt.get_attribute("value") != "-1"]

    # Loop through each Government sub-type
    for sub_type in govt_options:
        try:
            govt_dropdown = wait.until(EC.presence_of_element_located((By.NAME, "OptionGovtHosp")))
            select_govt = Select(govt_dropdown)
            select_govt.select_by_visible_text(sub_type)

            get_info_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Get Information']")))
            get_info_button.click()

            # Wait for hospital table to load
            hospital_table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[@class='tabBorder']")))
            table_rows = hospital_table.find_elements(By.XPATH, ".//tr[position()>1 and position()<last()]")

            updated_count = 0
            for row in table_rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 3:
                    hosp_name = cells[1].text.strip()
                    district = cells[2].text.strip()
                    # Match hospital by name, district, and type
                    mask = (
                        (hospital_with_govt_subtype_df["Hospital Name"].str.strip() == hosp_name) &
                        (hospital_with_govt_subtype_df["District"].str.strip() == district) &
                        (hospital_with_govt_subtype_df["Hospital Type"] == "Government")
                    )
                    if mask.any():
                        hospital_with_govt_subtype_df.loc[mask, "Government Sub-Type"] = sub_type
                        updated_count += mask.sum()

            print(f"Government sub-type '{sub_type}': {updated_count} hospitals updated.")

        except:
            print(f"No hospitals found for Government sub-type: {sub_type}. Continuing to next sub-type.")

except:
    print("No Government sub-type dropdown found or no hospitals for Government type.")

Government sub-type 'DH': 0 hospitals updated.
Government sub-type 'DHS': 35 hospitals updated.
Government sub-type 'DMER': 23 hospitals updated.
Government sub-type 'GH': 1 hospitals updated.
Government sub-type 'MCGM': 15 hospitals updated.
Government sub-type 'NMMC': 0 hospitals updated.
Government sub-type 'SDH100': 1 hospitals updated.
Government sub-type 'WH': 0 hospitals updated.


## Finalize Hospital DataFrame

We now clean and prepare the hospital DataFrame for storage:

1. Copy the DataFrame with Government sub-types.  
2. Rename columns to match the **database schema**.  
3. Strip whitespace from string columns.  
4. Remove duplicate hospital-district entries.  
5. Add a unique `hospital_id` as the index.  
6. Reorder columns and set `hospital_id` as the index.  
7. Display the first few rows to verify.


In [ ]:
# Copy the DataFrame
hospital_df_final = hospital_with_govt_subtype_df.copy()

# Rename columns to match database schema
hospital_df_final.rename(columns={
    "Hospital Name": "hospital_name",
    "District": "district",
    "Total Number of Beds": "total_beds",
    "HOSP DISP CODE": "hosp_disp_code",
    "Taluka": "taluka",
    "Address": "address",
    "Pincode": "pincode",
    "MCO Contact Number": "mco_contact_number",
    "Hospital Type": "hospital_type",
    "Government Sub-Type": "government_sub_type"
}, inplace=True)

# Strip whitespace from string columns
str_cols = ["hospital_name", "district", "hospital_type", "government_sub_type", "taluka", "address", "mco_contact_number", "hosp_disp_code"]
for col in str_cols:
    hospital_df_final[col] = hospital_df_final[col].astype(str).str.strip()

# Remove duplicate hospital-district entries
hospital_df_final = hospital_df_final.drop_duplicates(subset=["hospital_name", "district"]).reset_index(drop=True)

# Add unique hospital_id
hospital_df_final["hospital_id"] = hospital_df_final.index + 1

# Reorder columns and set hospital_id as index
column_order = ["hosp_disp_code","hospital_name","address","taluka","district","pincode","mco_contact_number","total_beds","hospital_type","government_sub_type"]
hospital_df_final = hospital_df_final[column_order]
hospital_df_final.set_index("hospital_id", inplace=True)

# Display first few rows
hospital_df_final.head()

,hosp_disp_code,hospital_name,address,taluka,district,pincode,mco_contact_number,total_beds,hospital_type,government_sub_type
hospital_id,,,,,,,,,,
1,IAAHC,INDRAVATI HOSPITAL & RESEARCH CENTRE,"Plot no. GHANSOLI VILLAGE, RCC HOUSE, ARJUNE W...",NA,THANE,400701,8452862559,30,Corporate,None
2,MPCT,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,"C7, Budhyadev Mandir Marg, Sector 4, Sanpada, ...",NA,THANE,400705,9702693695,100,Corporate,None
3,None,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,None,None,MUMBAI,None,None,15,Corporate,None
4,SANEI,SANJEEVANI HOSPITAL,"NEAR TAHSIL OFFICE, GOTMARE LAYOUT, BRAHMI KA...",NA,NAGPUR,441501,9860447504,30,Corporate,None
5,AGAHT,A & G HOSPITAL,Suchak House murbad road opp karnik road kalya...,NA,THANE,421301,9326114930,50,Corporate,None


In [190]:
hospital_df_final.to_csv("../data/maharashtra/hospital.csv", index=True)

## Scrape Hospital Service Categories

We now extract all **hospital service categories** from the portal:

1. Open the hospital info page.  
2. Locate the category (disease/service) dropdown.  
3. Get all options except the placeholder (`value=-1`).  
4. Create a DataFrame with `category_id` and `category_name`.  
5. Set `category_id` as the index.


In [ ]:
# Open the portal
driver.get(hospital_info_url)

# Locate the category dropdown and create a Select object
category_dropdown = wait.until(EC.presence_of_element_located((By.ID, "DiseaseM")))
select_category = Select(category_dropdown)

# Get all category names except the placeholder
category_options = [opt.text.strip() for opt in select_category.options if opt.get_attribute("value") != "-1"]

# Create a DataFrame for categories
category_df = pd.DataFrame({
    "category_id": range(1, len(category_options)+1),
    "category_name": category_options
}).set_index("category_id")

# Display the DataFrame
category_df

,category_name
category_id,
1,Burns (S13)
2,Cardiac And Cardiothoracic Surgery (S7)
3,Cardiology (M7)
4,Critical Care (M3)
5,Dermatology (M11)
6,ENT (S2)
7,Endocrinology (M13)
8,General Medicine (M16)
9,General Surgery (S1)


In [33]:
category_df.to_csv("../data/maharashtra/category.csv", index=True)

In [34]:
hospital_df_final = pd.read_csv("../data/maharashtra/hospital.csv", index_col=0)
category_df = pd.read_csv("../data/maharashtra/category.csv", index_col=0)

## Create Lookup Dictionaries

We create **lookup dictionaries** to quickly map:

- `(hospital_name, district)` → `hospital_id`  
- `category_name` → `category_id`  

These will help when creating the **hospital-category mapping table**.


In [ ]:
# Create a lookup for hospitals by (name, district)
hospital_lookup = {
    (row["hospital_name"].strip().lower(), row["district"].strip().lower()): hid
    for hid, row in hospital_df_final.iterrows()
}

# Create a lookup for categories by name
category_lookup = {
    row["category_name"].strip().lower(): cid
    for cid, row in category_df.iterrows()
}

## Initialize Hospital-Category Mapping

We prepare to create the **hospital-category mapping CSV**:

- `last_success_index` → Tracks the last successfully processed row (useful for resuming).  
- `file_path` → Path to save the mapping CSV.


In [45]:
last_success_index = -1
file_path = "../data/maharashtra/hospital_category.csv"

## Scrape Hospital-Category Mapping

We now create the **hospital-category mapping table**:

1. Loop through each service category.  
2. Select the category on the portal and click "Get Information".  
3. Extract hospital name, district, and last update date.  
4. If the hospital and category exist in the lookup dictionaries, add a mapping record.  
5. Append or create the `hospital_category.csv` file for each category.  
6. Track the `last_success_index` to resume in case of errors.


In [ ]:
# Open the portal
driver.get(hospital_info_url)

# Get all category names
category_dropdown = wait.until(EC.presence_of_element_located((By.ID, "DiseaseM")))
select_category = Select(category_dropdown)
category_texts = [str(opt.text).strip() for opt in select_category.options if opt.get_attribute("value") != "-1"]

# Loop through categories starting from last success
for idx in range(last_success_index + 1, len(category_texts)):
    hospital_category_records = []
    cat_text = category_texts[idx]
    try:
        # Select current category
        category_dropdown = wait.until(EC.presence_of_element_located((By.ID, "DiseaseM")))
        select_category = Select(category_dropdown)
        select_category.select_by_visible_text(cat_text)

        # Click "Get Information" button
        get_info_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Get Information']")))
        get_info_button.click()

        # Wait for hospital table and get rows
        hospital_table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[@class='tabBorder']")))
        table_rows = hospital_table.find_elements(By.XPATH, ".//tr[position()>1 and position()<last()]")

        valid_count = 0
        for row in table_rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 7:
                hosp_name = cells[1].text.strip()
                district = cells[2].text.strip()
                last_update = cells[6].text.strip()

                # Only include if last update exists
                if last_update != "--":
                    key = (hosp_name.lower(), district.lower())
                    hid = hospital_lookup.get(key)
                    cid = category_lookup.get(cat_text.strip().lower())
                    if hid is not None and cid is not None:
                        hospital_category_records.append({
                            "hospital_id": hid,
                            "category_id": cid
                        })
                        valid_count += 1

        print(f"[{idx}] Category '{cat_text}': {valid_count} hospitals found")

        # Save to CSV (append if exists, else create new)
        partial_df = pd.DataFrame(hospital_category_records)
        if os.path.exists(file_path):
            partial_df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            partial_df.to_csv(file_path, mode='w', header=True, index=False)

        # Update last success index
        last_success_index = idx

    except Exception as e:
        print(f"Error processing category '{cat_text}' at index {idx}: {e}") 
        break

[18] Category 'Mental Health Packages (M17)': 22 hospitals found
[19] Category 'Neonatal and Pediatric Medical Management (M6)': 376 hospitals found
[20] Category 'Nephrology (M8)': 742 hospitals found
[21] Category 'Neurology (M9)': 184 hospitals found
[22] Category 'Neurosurgery (S10)': 261 hospitals found
[23] Category 'OTHERS (OTH)': 0 hospitals found
[24] Category 'Ophthalmology Surgery (S3)': 142 hospitals found
[25] Category 'Orthopedic Surgery And Procedures (S5)': 725 hospitals found
[26] Category 'Pediatric Cancer (S21)': 2 hospitals found
[27] Category 'Pediatric Surgery (S8)': 86 hospitals found
[28] Category 'Physiotherapy (M20)': 0 hospitals found
[29] Category 'Plastic Surgery (S12)': 68 hospitals found
[30] Category 'Polytrauma (S14)': 198 hospitals found
[31] Category 'Prosthesis and Orthosis (S15)': 2 hospitals found
[32] Category 'Pulmonology (M10)': 332 hospitals found
[33] Category 'Radiation Oncology (M2)': 65 hospitals found
[34] Category 'Rheumatology (M12)': 19

# Maharashtra Healthcare Population Data – Planning & Initial Setup

**Objective:**  
Prepare population datasets for Maharashtra to support healthcare inequality analysis and align them with the hospital dataset.

**Key Points about Hospital Data:**  
- `hospital.csv` already uses **modern district names**, including some that are **not yet officially recognized**.  
- For consistency with official government data, **we will keep only current official district names** and map or exclude any unofficial ones.  
- This ensures smooth integration with population data from Census 2011.

**Planned Tasks:**  

1. **Population Data Collection**  
   - Source population data from the [Census of India 2011 – Population Search PCA Indicators](https://censusindia.gov.in/census.website/en/data/data-visualizations/PopulationSearch_PCA_Indicators).  
   - Extract population statistics for all districts and sub-districts (talukas).  
   - Handle special cases like the **Thane-Palghar split**, aggregating sub-district populations to match current official districts.  

2. **Data Cleaning and Standardization**  
   - Standardize population dataset district names to match the **official district names** in `hospital.csv`.  
   - Remove or map any unofficial/new district names in hospital data as needed.  

3. **Data Storage**  
   - Save the processed population data as `population.csv` for downstream analysis.  
   - This CSV will contain **district-level total and children populations**, with separate counts for **Persons, Males, and Females**.


In [53]:
df = pd.read_csv("../data/maharashtra/hospital.csv", index_col=0)

In [54]:
sorted(df["district"].unique())

['AHILYANAGAR',
 'AKOLA',
 'AMRAVATI',
 'BARSHI',
 'BEED',
 'BHANDARA',
 'BULDHANA',
 'CHANDRAPUR',
 'CHATTRAPATHI SAMBHAJI NAGAR',
 'DHARASHIV',
 'DHULE',
 'GADCHIROLI',
 'GONDIYA',
 'HINGOLI',
 'JALGAON',
 'JALNA',
 'KOLHAPUR',
 'LATUR',
 'MUMBAI',
 'MUMBAI SUBURBAN',
 'NAGPUR',
 'NANDED',
 'NANDURBAR',
 'NASHIK',
 'PALGHAR',
 'PANDHARPUR',
 'PANVEL',
 'PARBHANI',
 'PUNE',
 'RAIGAD',
 'RATNAGIRI',
 'SANGLI',
 'SATARA',
 'SHIRDI',
 'SHIRPUR',
 'SINDHUDURG',
 'SOLAPUR',
 'THANE',
 'WARDHA',
 'WASHIM',
 'YAVATMAL']

In [49]:
len(df["district"].unique())

41

**Dataset vs Census 2011 Districts**

- The hospital dataset currently contains **41 districts**.  
- The Census 2011 dataset, which is being used as a reference for population data, contains only **35 districts**.  
- Additionally, some district names differ between the two datasets, which may require **name mapping or reconciliation** to ensure consistency when analyzing healthcare availability per district.


**Standardizing District Names**

This cell updates district names in the hospital dataset to **current official district names**.  
- Talukas or older names are mapped to the correct current districts.  
- Any district not in the mapping keeps its original name.


In [55]:
district_mapping = {
    "CHATTRAPATHI SAMBHAJI NAGAR": "CHHATRAPATI SAMBHAJINAGAR",
    "BARSHI": "SOLAPUR",         # Barshi is a taluka in Solapur
    "PANDHARPUR": "SANGLI",      # Pandharpur is a taluka in Sangli
    "PANVEL": "RAIGAD",          # Panvel is a taluka in Raigad
    "SHIRDI": "AHILYANAGAR",      # Shirdi is a taluka in Ahmednagar
    "SHIRPUR": "DHULE"           # Shirpur is a taluka in Dhule
}

df['district'] = df['district'].apply(lambda x: district_mapping.get(x, x))

In [56]:
len(df["district"].unique())

36

In [57]:
sorted(df["district"].unique())

['AHILYANAGAR',
 'AKOLA',
 'AMRAVATI',
 'BEED',
 'BHANDARA',
 'BULDHANA',
 'CHANDRAPUR',
 'CHHATRAPATI SAMBHAJINAGAR',
 'DHARASHIV',
 'DHULE',
 'GADCHIROLI',
 'GONDIYA',
 'HINGOLI',
 'JALGAON',
 'JALNA',
 'KOLHAPUR',
 'LATUR',
 'MUMBAI',
 'MUMBAI SUBURBAN',
 'NAGPUR',
 'NANDED',
 'NANDURBAR',
 'NASHIK',
 'PALGHAR',
 'PARBHANI',
 'PUNE',
 'RAIGAD',
 'RATNAGIRI',
 'SANGLI',
 'SATARA',
 'SINDHUDURG',
 'SOLAPUR',
 'THANE',
 'WARDHA',
 'WASHIM',
 'YAVATMAL']

In [58]:
df.to_csv("../data/maharashtra/hospital.csv", index=True)

In [116]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 25)

**Handling Thane-Palghar District Split Using Census 2011 Data**

- **Background:**  
  - The **2011 Census of India** lists only **Thane** district.  
  - In the current official administrative structure, **Thane** has been bifurcated into **Thane** and **Palghar**.  
- **Challenge:**  
  - Hospital data uses current district names, including **Palghar**, while population data from 2011 exists only for the original Thane district.  
- **Solution:**  
  1. Obtain population data for **sub-districts (talukas)** from the [Census of India 2011](https://censusindia.gov.in/census.website/en/data/data-visualizations/PopulationSearch_PCA_Indicators) or official state sources.  
  2. Use the scraped `district_mapping` (from [IGOD Portal](https://igod.gov.in/sg/MH/E042/organizations)) to determine which sub-districts now belong to **Thane** and which to **Palghar**.  
  3. Sum the populations of the sub-districts corresponding to each current district to estimate populations for **Thane** and **Palghar** separately.  
- **Use Case:**  
  - This ensures accurate **per-district healthcare analysis** using current district boundaries, despite the census data using older boundaries.


In [ ]:
# Stores results in `district_mapping` with district as key and list of sub-districts as value.
district_mapping = {}
target_districts = ["THANE", "PALGHAR"]

driver.get("https://igod.gov.in/sg/MH/E042/organizations")

# Scroll until all rows are loaded
prev_count = 0
while True:
    rows = driver.find_elements(By.XPATH, "//div[contains(@class,'search-result-row')]")
    curr_count = len(rows)
    if curr_count == prev_count:
        break
    prev_count = curr_count
    driver.execute_script("arguments[0].scrollIntoView();", rows[-1])
    time.sleep(1)

wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'search-row')]//div[contains(@class,'search-result-row')]"))) 
rows = driver.find_elements(By.XPATH, "//div[@class='search-content']//div[contains(@class,'search-result-row')]")

# Iterate through rows and extract sub-districts for target districts
for i in range(len(rows)):
    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'search-result-row')]")))
    rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'search-result-row')]")
    row = rows[i]

    district_name = row.find_element(By.CSS_SELECTOR, "a.search-title").text.strip().upper()

    if district_name in target_districts:
        sub_district_link = row.find_element(By.XPATH, ".//a[contains(@href,'sub_districts')]")
        sub_district_link.click()
        
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'search-result-row')]//div[contains(@class,'search-title')]")))
            sub_rows = driver.find_elements(By.XPATH, "//div[contains(@class,'search-result-row')]//div[contains(@class,'search-title')]")
            sub_districts = [s.text.strip().upper() for s in sub_rows]
        except:
            sub_districts = []

        district_mapping[district_name] = sub_districts

        driver.back()

In [152]:
district_mapping

{'PALGHAR': ['DAHANU',
  'JAWHAR',
  'MOKHADA',
  'PALGHAR',
  'TALASARI',
  'VASAI',
  'VIKRAMGAD',
  'WADA'],
 'THANE': ['AMBARNATH',
  'BHIWANDI',
  'KALYAN',
  'MURBAD',
  'SHAHAPUR',
  'THANE',
  'ULHASNAGAR']}

**Population Data Source – Census 2011**

- **Purpose:** All population data for the EquiHealth project is sourced from the **Census of India 2011**, accessible via the [Population Search PCA Indicators](https://censusindia.gov.in/census.website/en/data/data-visualizations/PopulationSearch_PCA_Indicators) portal.  
- **Details:**  
  - Provides population statistics at **district** and **sub-district (taluka)** levels.  
  - Enables accurate estimation of populations for **current districts**, including those formed after 2011 (e.g., Palghar, bifurcated from Thane).  
  - Data can be aggregated from sub-districts to match current administrative boundaries for analysis.  
- **Use Case in EquiHealth:**  
  - Comparing **hospital availability per population** across districts.  
  - Handling district splits by summing sub-district populations to approximate current district populations.  
- **Source:** [Census of India 2011 – Population Search PCA Indicators](https://censusindia.gov.in/census.website/en/data/data-visualizations/PopulationSearch_PCA_Indicators)


In [167]:
census_url = "https://censusindia.gov.in/census.website/en/data/data-visualizations/PopulationSearch_PCA_Indicators"

**Fetching District List from Census 2011 Portal**

- **Purpose:** Access the Census 2011 [Population Search PCA Indicators](https://censusindia.gov.in/census.website/en/data/data-visualizations/PopulationSearch_PCA_Indicators) portal to retrieve the list of districts for Maharashtra.  
- **Steps:**  
  1. Load the Census portal using Selenium.  
  2. Switch to the iframe containing the population charts and dropdowns.  
  3. Select **Maharashtra** from the "State/UT" dropdown.  
  4. Extract district names from the "District" dropdown, ignoring the 'ALL' option.  
- **Output:** A list of district names in Maharashtra, which will be used for population data extraction per district.


In [ ]:
driver.get(census_url)

# Switch to iframe containing the population chart and dropdowns
iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#chart-iframe iframe")))
driver.switch_to.frame(iframe)

# Select the state as Maharashtra
state_select_el = wait.until(EC.element_to_be_clickable((By.XPATH, '//label[text()="State/UT"]/following-sibling::select')))
state_dropdown = Select(state_select_el)
state_dropdown.select_by_visible_text("Maharashtra")

time.sleep(1)

# Get the district dropdown options (excluding 'ALL')
district_select_el = wait.until(EC.presence_of_element_located((By.XPATH, '//label[text()="District"]/following-sibling::select')))
district_dropdown = Select(district_select_el)

district_list = [opt.text.strip() for opt in district_dropdown.options if opt.text.strip() and opt.text.strip().upper() != "ALL"]

**Extracting Sub-Districts for Thane from Census 2011 Portal**

- **Purpose:** Retrieve all sub-districts (talukas) under **Thane** from the Census 2011 portal to handle the Thane-Palghar district split.  
- **Steps:**  
  1. Load the [Census 2011 Population Search PCA Indicators](https://censusindia.gov.in/census.website/en/data/data-visualizations/PopulationSearch_PCA_Indicators) portal.  
  2. Switch to the iframe containing dropdowns for state, district, and sub-district.  
  3. Select **Maharashtra** as the state.  
  4. Select **Thane** as the district.  
  5. Extract all sub-district names under Thane, excluding the 'ALL' option.  
- **Output:** `thane_subdistrict_list` contains the list of talukas under Thane, which can be used to sum populations for the current districts **Thane** and **Palghar**.


In [ ]:
target_district = "Thane"

driver.get(census_url)

# Switch to iframe containing the population chart and dropdowns
iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#chart-iframe iframe")))
driver.switch_to.frame(iframe)

# Select Maharashtra as the state
state_select_el = wait.until(EC.element_to_be_clickable((By.XPATH, '//label[text()="State/UT"]/following-sibling::select')))
state_dropdown = Select(state_select_el)
state_dropdown.select_by_visible_text("Maharashtra")

time.sleep(1)

# Select Thane as the district
district_select_el = wait.until(EC.presence_of_element_located((By.XPATH, '//label[text()="District"]/following-sibling::select')))
district_dropdown = Select(district_select_el)
district_dropdown.select_by_visible_text(target_district)
time.sleep(1)

# Extract all sub-districts under Thane
subdistrict_select_el = wait.until(EC.presence_of_element_located((By.XPATH, '//label[text()="Sub-District"]/following-sibling::select')))
subdistrict_dropdown = Select(subdistrict_select_el)
wait.until(lambda d: len([opt for opt in subdistrict_dropdown.options if opt.text.strip().upper() != "ALL"]) > 0)

thane_subdistrict_list = [opt.text.strip() for opt in subdistrict_dropdown.options if opt.text.strip() and opt.text.strip().upper() != "ALL"]

In [161]:
district_list

['Ahmadnagar',
 'Akola',
 'Amravati',
 'Aurangabad',
 'Bhandara',
 'Bid',
 'Buldana',
 'Chandrapur',
 'Dhule',
 'Gadchiroli',
 'Gondiya',
 'Hingoli',
 'Jalgaon',
 'Jalna',
 'Kolhapur',
 'Latur',
 'Mumbai',
 'Mumbai Suburban',
 'Nagpur',
 'Nanded',
 'Nandurbar',
 'Nashik',
 'Osmanabad',
 'Parbhani',
 'Pune',
 'Raigarh',
 'Ratnagiri',
 'Sangli',
 'Satara',
 'Sindhudurg',
 'Solapur',
 'Thane',
 'Wardha',
 'Washim',
 'Yavatmal']

In [162]:
thane_subdistrict_list

['Ambarnath',
 'Bhiwandi',
 'Dahanu',
 'Jawhar',
 'Kalyan',
 'Mokhada',
 'Murbad',
 'Palghar',
 'Shahapur',
 'Talasari',
 'Thane',
 'Ulhasnagar',
 'Vada',
 'Vasai',
 'Vikramgad']

In [169]:
population_data = {}

**Scraping Population Data for Maharashtra Districts (Excluding Thane)**

- **Purpose:** Extract population details for all districts in Maharashtra except **Thane** from the [Census of India 2011 Population Search PCA Indicators](https://censusindia.gov.in/census.website/en/data/data-visualizations/PopulationSearch_PCA_Indicators) portal.  
- **Steps:**  
  1. Load the portal and switch to the iframe containing dropdowns and population table.  
  2. Select **Maharashtra** as the state.  
  3. Loop through all district options except **Thane**.  
  4. For each district, wait for the population table (`popTable`) to load.  
  5. Extract population data for **Total population** and **Children population**, separating **Persons, Males, Females**.  
  6. Store the data in the `population_data` dictionary with the district name as the key.  
- **Output:** `population_data` dictionary containing population counts for each district in Maharashtra (excluding Thane), ready for healthcare analysis.


In [ ]:
driver.get(census_url)

# Switch to iframe containing the population chart and dropdowns
iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#chart-iframe iframe")))
driver.switch_to.frame(iframe)

# Select Maharashtra as the state
state_select_el = wait.until(EC.element_to_be_clickable((By.XPATH, '//label[text()="State/UT"]/following-sibling::select')))
Select(state_select_el).select_by_visible_text("Maharashtra")
time.sleep(1)

# Get district options
district_select_el = wait.until(EC.presence_of_element_located((By.XPATH, '//label[text()="District"]/following-sibling::select')))
district_dropdown = Select(district_select_el)
district_options = [opt.text.strip() for opt in district_dropdown.options if opt.text.strip() and opt.text.strip().upper() != "ALL"]

# Iterate through districts and extract population data (skip Thane)
for district in district_options:
    if district.upper() == "THANE":
        continue

    district_dropdown.select_by_visible_text(district)
    time.sleep(1)

    table = wait.until(EC.presence_of_element_located((By.ID, "popTable")))
    rows = table.find_elements(By.TAG_NAME, "tr")

    population = {"Total": {}, "Children": {}}

    tbody_rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")

    if len(tbody_rows) >= 2:
        total_row = tbody_rows[0].find_elements(By.TAG_NAME, "td")
        child_row = tbody_rows[1].find_elements(By.TAG_NAME, "td")

        if len(total_row) >= 4 and len(child_row) >= 4:
            population_data[district] = {
                "Total": {
                    "Persons": int(total_row[1].text.replace(",", "")),
                    "Males": int(total_row[2].text.replace(",", "")),
                    "Females": int(total_row[3].text.replace(",", ""))
                },
                "Children": {
                    "Persons": int(child_row[1].text.replace(",", "")),
                    "Males": int(child_row[2].text.replace(",", "")),
                    "Females": int(child_row[3].text.replace(",", ""))
                }
            }
        else:
            print(f"Row cells missing for district: {district}")
    else:
        print(f"Not enough rows in table for district: {district}")

**Splitting Thane Population Between Thane and Palghar**

- **Purpose:** Aggregate population counts from **sub-districts (talukas)** under the old Thane district to estimate current populations for **Thane** and **Palghar**.  
- **Steps:**  
  1. Load the [Census 2011 Population Search PCA Indicators](https://censusindia.gov.in/census.website/en/data/data-visualizations/PopulationSearch_PCA_Indicators) portal.  
  2. Switch to the iframe and select **Maharashtra** as the state and **Thane** as the district.  
  3. Extract all sub-district options under Thane.  
  4. Initialize `population_data` entries for both Thane and Palghar with zeros.  
  5. Iterate through sub-districts, fetch population data from the table, and assign counts to **Thane** or **Palghar** based on the `district_mapping` dictionary.  
- **Output:** `population_data` now contains aggregated **Total** and **Children** populations for both current districts, enabling accurate per-district healthcare analysis despite historical district boundaries.


In [ ]:
driver.get(census_url)

# Switch to iframe containing the population chart and dropdowns
iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#chart-iframe iframe")))
driver.switch_to.frame(iframe)

# Select Maharashtra as the state
state_select_el = wait.until(EC.element_to_be_clickable((By.XPATH, '//label[text()="State/UT"]/following-sibling::select')))
Select(state_select_el).select_by_visible_text("Maharashtra")
time.sleep(1)

# Select Thane as the district
district_select_el = wait.until(EC.presence_of_element_located((By.XPATH, '//label[text()="District"]/following-sibling::select')))
district_dropdown = Select(district_select_el)
district_dropdown.select_by_visible_text("Thane")
time.sleep(1)

# Extract sub-districts under Thane
subdistrict_select_el = wait.until(EC.presence_of_element_located((By.XPATH, '//label[text()="Sub-District"]/following-sibling::select')))
subdistrict_dropdown = Select(subdistrict_select_el)
subdistrict_options = [opt.text.strip() for opt in subdistrict_dropdown.options if opt.text.strip() and opt.text.strip().upper() != "ALL"]

# Initialize population counts for Thane and Palghar
population_data["THANE"] = {"Total": {"Persons": 0, "Males": 0, "Females": 0}, "Children": {"Persons": 0, "Males": 0, "Females": 0}}
population_data["PALGHAR"] = {"Total": {"Persons": 0, "Males": 0, "Females": 0}, "Children": {"Persons": 0, "Males": 0, "Females": 0}}

# Aggregate population by sub-district to assign to current districts
for subdistrict in subdistrict_options:
    subdistrict_dropdown.select_by_visible_text(subdistrict)
    time.sleep(1)

    table = wait.until(EC.presence_of_element_located((By.ID, "popTable")))
    tbody_rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")

    if len(tbody_rows) >= 2:
        total_row = tbody_rows[0].find_elements(By.TAG_NAME, "td")
        child_row = tbody_rows[1].find_elements(By.TAG_NAME, "td")

        if len(total_row) >= 4 and len(child_row) >= 4:
            # Determine which district this sub-district belongs to using district_mapping
            for dist, sub_list in district_mapping.items():
                if subdistrict.upper() in sub_list:
                    population_data[dist]["Total"]["Persons"] += int(total_row[1].text.replace(",", ""))
                    population_data[dist]["Total"]["Males"] += int(total_row[2].text.replace(",", ""))
                    population_data[dist]["Total"]["Females"] += int(total_row[3].text.replace(",", ""))
                    population_data[dist]["Children"]["Persons"] += int(child_row[1].text.replace(",", ""))
                    population_data[dist]["Children"]["Males"] += int(child_row[2].text.replace(",", ""))
                    population_data[dist]["Children"]["Females"] += int(child_row[3].text.replace(",", ""))
                    break
        else:
            print(f"Row cells missing for subdistrict: {subdistrict}")
    else:
        print(f"Not enough rows in table for subdistrict: {subdistrict}")

In [172]:
len(population_data)

36

**Creating Population DataFrame**

- **Purpose:** Convert the aggregated population data stored in the `population_data` dictionary into a **Pandas DataFrame** for easy analysis and visualization.  
- **Steps:**  
  1. Iterate through each district and extract **Total** and **Children** population counts (Persons, Males, Females).  
  2. Create a list of dictionaries (`data_rows`), each representing a district.  
  3. Convert the list into a DataFrame `df_population`.  
- **Output:** `df_population` contains one row per district with detailed population statistics, ready for integration with hospital data to compute metrics like **hospital beds per capita**.


In [177]:
data_rows = []

for district, values in population_data.items():
    district_upper = district.upper()
    row = {
        "district": district_upper,
        "total_persons": values["Total"]["Persons"],
        "total_males": values["Total"]["Males"],
        "total_females": values["Total"]["Females"],
        "children_persons": values["Children"]["Persons"],
        "children_males": values["Children"]["Males"],
        "children_females": values["Children"]["Females"]
    }
    data_rows.append(row)

df_population = pd.DataFrame(data_rows)

df_population

,district,total_persons,total_males,total_females,children_persons,children_males,children_females
0,AHMADNAGAR,4543159,2342825,2200334,556014,300238,255776
1,AKOLA,1813906,932334,881572,211080,110372,100708
2,AMRAVATI,2888445,1480768,1407677,311391,160934,150457
3,AURANGABAD,3701282,1924469,1776813,532659,286721,245938
4,BHANDARA,1200334,605520,594814,126025,64626,61399
5,BID,2585049,1349106,1235943,352890,195245,157645
6,BULDANA,2586258,1337560,1248698,332125,179072,153053
7,CHANDRAPUR,2204307,1123834,1080473,231316,118471,112845
8,DHULE,2050862,1054031,996831,273507,144121,129386
9,GADCHIROLI,1072942,541328,531614,120272,61320,58952


In [179]:
sorted(df_population["district"])

['AHMADNAGAR',
 'AKOLA',
 'AMRAVATI',
 'AURANGABAD',
 'BHANDARA',
 'BID',
 'BULDANA',
 'CHANDRAPUR',
 'DHULE',
 'GADCHIROLI',
 'GONDIYA',
 'HINGOLI',
 'JALGAON',
 'JALNA',
 'KOLHAPUR',
 'LATUR',
 'MUMBAI',
 'MUMBAI SUBURBAN',
 'NAGPUR',
 'NANDED',
 'NANDURBAR',
 'NASHIK',
 'OSMANABAD',
 'PALGHAR',
 'PARBHANI',
 'PUNE',
 'RAIGARH',
 'RATNAGIRI',
 'SANGLI',
 'SATARA',
 'SINDHUDURG',
 'SOLAPUR',
 'THANE',
 'WARDHA',
 'WASHIM',
 'YAVATMAL']

**Standardizing District Names in Population Data**

- **Purpose:** Ensure the district names in `df_population` match the **current official district names** used in hospital datasets.  
- **Steps:**  
  1. Define a `district_name_mapping` dictionary to map old names to current names.  
  2. Replace district names in the DataFrame using this mapping.  
  3. Set `district` as the **index** to facilitate merging with hospital data and other analyses.  
- **Result:** `df_population` now uses standardized district names and is ready for integration with other datasets for healthcare inequality analysis.


In [180]:
district_name_mapping = {
    "AHMADNAGAR": "AHILYANAGAR",
    "AURANGABAD": "CHHATRAPATI SAMBHAJINAGAR",
    "BID": "BEED",
    "BULDANA": "BULDHANA",
    "OSMANABAD": "DHARASHIV",
    "RAIGARH": "RAIGAD"
}

df_population["district"] = df_population["district"].replace(district_name_mapping)

df_population.set_index("district", inplace=True)

In [181]:
df_population.to_csv("../data/maharashtra/population.csv")

### State Latitude and Longitude Table

We will prepare a table containing all Indian states and union territories with their corresponding **latitude** and **longitude** coordinates.  

The data will be sourced from the Kaggle dataset: [States & UTs of India (Updated Data w/ Coordinates)](https://www.kaggle.com/datasets/satyampd/states-uts-of-indiaupdated-data-w-coordinates).

The purpose of this table is to create a **reference for state-level coordinates**, which we will later use to **map `state_id` to districts** in the district-level dataset. This will allow each district to be associated with its corresponding state for further analysis and mapping.


In [39]:
state_df = pd.read_csv("../data/India States-Uts.csv")

state_df.rename(columns={
    'State/UT': 'state_name',
    'Latitude': 'latitude',
    'Longitude': 'longitude'
}, inplace=True)

state_df['state_name'] = state_df['state_name'].str.upper()

state_df['state_id'] = range(1, len(state_df) + 1)

state_df.set_index('state_id', inplace=True)

state_df

,state_name,latitude,longitude
state_id,,,
1,ANDAMAN AND NICOBAR ISLANDS,11.7401,92.6586
2,ANDHRA PRADESH,15.9129,79.7400
3,ARUNACHAL PRADESH,28.2180,94.7278
4,ASSAM,31.1048,77.1734
5,BIHAR,25.0961,85.3131
6,CHHATTISGARH,21.2787,81.8661
7,DADRA AND NAGAR HAVELI AND DAMAN AND DIU,20.1809,73.0169
8,DELHI,28.7041,77.1025
9,GOA,15.2993,74.1240


In [41]:
state_df.to_csv("../data/state.csv")

## Assigning Latitude and Longitude to Districts

In this task, we aim to enhance the district-level dataset by adding **geographical coordinates** (latitude and longitude) for each district in Maharashtra.  

- We use the [**All Indian Pin Codes 03-03-24** dataset](https://www.kaggle.com/datasets/divanshu22/all-indian-pin-codes-03-03-24) from Kaggle, which contains postal codes along with their corresponding locations across India.  
- By filtering for Maharashtra and aggregating coordinates at the district level, we can compute **average latitude and longitude** for each district.  
- These coordinates will be included in the `district` table of our database, enabling **mapping, spatial analysis, and integration with hospital locations** in later tasks.


In [46]:
district_df = pd.read_csv("../data/maharashtra/population.csv")
district_df.head()

,district,total_persons,total_males,total_females,children_persons,children_males,children_females
0,AHILYANAGAR,4543159,2342825,2200334,556014,300238,255776
1,AKOLA,1813906,932334,881572,211080,110372,100708
2,AMRAVATI,2888445,1480768,1407677,311391,160934,150457
3,CHHATRAPATI SAMBHAJINAGAR,3701282,1924469,1776813,532659,286721,245938
4,BHANDARA,1200334,605520,594814,126025,64626,61399


### All Indian Pin Codes 03-03-24

This dataset contains information about postal codes across India, including district names, state names, and latitude/longitude coordinates.  

Source: [Kaggle: All Indian Pin Codes 03-03-24](https://www.kaggle.com/datasets/divanshu22/all-indian-pin-codes-03-03-24)

We will use this dataset to assign **latitude and longitude** values to districts.


In [75]:
pincode_df = pd.read_csv("../data/pincode.csv", dtype={"Pincode": "string"}, low_memory=False)

### Converting Latitude and Longitude to Numeric

In this cell, the `Latitude` and `Longitude` columns of the pin code dataset are converted to numeric values using `pd.to_numeric()`.  
Non-numeric or invalid values are coerced to `NaN`.  

This ensures that the coordinates can be used later for computations, mapping, or linking districts to geographic locations.


In [76]:
pincode_df["Latitude"] = pd.to_numeric(pincode_df["Latitude"], errors="coerce")
pincode_df["Longitude"] = pd.to_numeric(pincode_df["Longitude"], errors="coerce")

In [77]:
pincode_df.head()

,CircleName,RegionName,DivisionName,OfficeName,Pincode,OfficeType,Delivery,District,StateName,Latitude,Longitude
0,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Peddakotla B.O,515631,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.568900,77.856240
1,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Pinnadhari B.O,515631,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.528100,77.857014
2,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Yerraguntapalle B.O,515631,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.561111,77.857150
3,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Obulareddipalli B.O,515581,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.248800,78.258800
4,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Odulapalli B.O,515581,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.245550,78.247700


### Extracting District Coordinates for Maharashtra

This cell filters the pin code dataset to only include entries from **Maharashtra**.  
Rows with missing latitude or longitude values are removed to ensure accuracy.  

Next, the dataset is grouped by district, and the mean latitude and longitude are calculated for each district.  
These coordinates will later be used to assign geographic locations to districts in the database.

Finally, the columns are renamed to match the district table schema: `district_name`, `latitude`, and `longitude`.


In [50]:
pincode_maha = pincode_df[pincode_df["StateName"].str.upper() == "MAHARASHTRA"].copy()
pincode_maha = pincode_maha.dropna(subset=["Latitude", "Longitude"])

district_coords = pincode_maha.groupby("District", as_index=False).agg({"Latitude": "mean", "Longitude": "mean"})

district_coords.rename(columns={"District": "district_name", "Latitude": "latitude", "Longitude": "longitude"}, inplace=True)
district_coords.head()

,district_name,latitude,longitude
0,AHMEDNAGAR,19.265435,74.637576
1,AKOLA,20.765138,77.054701
2,AMRAVATI,21.12858,77.648207
3,AURANGABAD,19.944847,75.41926
4,BEED,18.957225,75.625726


### Standardizing District Names for Matching

This cell prepares district names in both the district DataFrame (`district_df`) and the coordinates DataFrame (`district_coords`) for merging:

1. **Normalization:** Converts names to uppercase and strips leading/trailing spaces.
2. **Manual Mapping:** Corrects discrepancies between different naming conventions using a mapping dictionary.

The resulting `district_clean` column ensures consistent names across datasets for accurate merging later.


In [51]:
district_df["district_clean"] = district_df["district"].str.upper().str.strip()
district_coords["district_clean"] = district_coords["district_name"].str.upper().str.strip()

district_name_mapping = {
    "AHMEDNAGAR": "AHILYANAGAR",
    "AURANGABAD": "CHHATRAPATI SAMBHAJINAGAR",
    "GONDIA": "GONDIYA",
    "OSMANABAD": "DHARASHIV"
}

district_coords["district_clean"] = district_coords["district_clean"].replace(district_name_mapping)

### Merging District Data with Coordinates

This cell combines the district demographic data (`district_df`) with the latitude and longitude information (`district_coords`) using the standardized `district_clean` column.

- **Merge Type:** Left join to keep all districts in `district_df` even if coordinates are missing.
- **Result:** `district_info` now contains all district-level demographic information along with their approximate geographic coordinates.


In [53]:
district_info = district_df.merge(district_coords, on="district_clean", how="left")

In [58]:
district_info.head()

,district,total_persons,total_males,total_females,children_persons,children_males,children_females,district_clean,district_name,latitude,longitude
0,AHILYANAGAR,4543159,2342825,2200334,556014,300238,255776,AHILYANAGAR,AHMEDNAGAR,19.265435,74.637576
1,AKOLA,1813906,932334,881572,211080,110372,100708,AKOLA,AKOLA,20.765138,77.054701
2,AMRAVATI,2888445,1480768,1407677,311391,160934,150457,AMRAVATI,AMRAVATI,21.12858,77.648207
3,CHHATRAPATI SAMBHAJINAGAR,3701282,1924469,1776813,532659,286721,245938,CHHATRAPATI SAMBHAJINAGAR,AURANGABAD,19.944847,75.41926
4,BHANDARA,1200334,605520,594814,126025,64626,61399,BHANDARA,BHANDARA,21.179723,79.252516


### Finalizing District Table

This cell prepares the district-level dataset for the database:

- Removes temporary columns used for merging (`district_clean`, `district_name` from coordinates).
- Renames the original `district` column back to `district_name`.
- Adds a `state_name` column with value `"MAHARASHTRA"`.
- Resets the index and creates a `district_id` as a primary key.
- Reorders columns to match the database schema for districts.
- **Result:** `district_info` is now ready to be used as the `district` table in the database.


In [59]:
district_info = district_info.drop(columns=["district_clean", "district_name"])

district_info = district_info.rename(columns={"district": "district_name"})

district_info["state_name"] = "MAHARASHTRA"

district_info = district_info.reset_index(drop=True)
district_info["district_id"] = district_info.index + 1
district_info = district_info.set_index("district_id")

district_info = district_info[[
    "district_name",
    "state_name",
    "latitude",
    "longitude",
    "total_persons",
    "total_males",
    "total_females",
    "children_persons",
    "children_males",
    "children_females"
]]

In [61]:
district_info.to_csv("../data/maharashtra/district.csv")

### Assigning `state_id` to Districts

We will assign the `state_id` from the **state-level table (`state_df`)** to the **district-level table (`district_df`)**.  

This mapping will allow each district to be associated with its corresponding state, making it easier to perform **joins, mappings, and geographic analyses**.


In [35]:
state_df = pd.read_csv("../data/state.csv", index_col=0)
district_df = pd.read_csv("../data/maharashtra/district.csv", index_col=0)

In [ ]:
maharashtra_id = state_df[state_df['state'] == 'MAHARASHTRA'].index[0]
district_df['state_id'] = maharashtra_id
district_df.drop(columns=['state_name'], inplace=True)

cols = district_df.columns.tolist()
cols.remove('state_id')
cols.insert(1, 'state_id')
district_df = district_df[cols]

district_df

,district,state_id,latitude,longitude,total_persons,total_males,total_females,children_persons,children_males,children_females
district_id,,,,,,,,,,
1,AHILYANAGAR,18,19.265435,74.637576,4543159,2342825,2200334,556014,300238,255776
2,AKOLA,18,20.765138,77.054701,1813906,932334,881572,211080,110372,100708
3,AMRAVATI,18,21.128580,77.648207,2888445,1480768,1407677,311391,160934,150457
4,CHHATRAPATI SAMBHAJINAGAR,18,19.944847,75.419260,3701282,1924469,1776813,532659,286721,245938
5,BHANDARA,18,21.179723,79.252516,1200334,605520,594814,126025,64626,61399
6,BEED,18,18.957225,75.625726,2585049,1349106,1235943,352890,195245,157645
7,BULDHANA,18,20.489346,76.196112,2586258,1337560,1248698,332125,179072,153053
8,CHANDRAPUR,18,20.680431,78.567138,2204307,1123834,1080473,231316,118471,112845
9,DHULE,18,21.090043,74.480471,2050862,1054031,996831,273507,144121,129386


In [37]:
district_df.to_csv("../data/maharashtra/district.csv")

## Assigning `district_id` to Hospitals

In this step, we replace the `district` column in `hospital.csv` with a proper `district_id` referencing the `district` table.  

- We use the previously created `district.csv` to create a lookup dictionary mapping district names to their IDs.  
- Hospital entries are then linked to their respective `district_id`.  
- Extra or temporary columns used for mapping are removed to conform to the database schema.  

This ensures that the `Hospital` table has a foreign key reference to `District`, enabling relational integrity and easier aggregation of data by district.


In [89]:
hospital_df = pd.read_csv("../data/maharashtra/hospital.csv", index_col=0)
hospital_df.head()

,hospital_name,address,pincode,mco_contact_number,total_beds,hospital_type,government_sub_type,district_id
hospital_id,,,,,,,,
1,INDRAVATI HOSPITAL & RESEARCH CENTRE,"Plot no. GHANSOLI VILLAGE, RCC HOUSE, ARJUNE W...",400701.0,8.452863e+09,30,Corporate,NaN,35
2,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,"C7, Budhyadev Mandir Marg, Sector 4, Sanpada, ...",400705.0,9.702694e+09,100,Corporate,NaN,35
3,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,NaN,NaN,NaN,15,Corporate,NaN,17
4,SANJEEVANI HOSPITAL,"NEAR TAHSIL OFFICE, GOTMARE LAYOUT, BRAHMI KA...",441501.0,9.860448e+09,30,Corporate,NaN,19
5,A & G HOSPITAL,Suchak House murbad road opp karnik road kalya...,421301.0,9.326115e+09,50,Corporate,NaN,35


## Creating District Lookup for Hospital Assignment

We create a **lookup dictionary** to map each district's name to its `district_id`.  

- This will be used to link hospitals to their corresponding district.  
- The `district_name` is cleaned by converting to **lowercase** and stripping whitespace to ensure consistent matching.


In [91]:
district_df = pd.read_csv("../data/maharashtra/district.csv", index_col=0)

# Create a lookup dictionary for districts
district_lookup = {
    row["district_name"].strip().lower(): did
    for did, row in district_df.iterrows()
}

## Assigning `district_id` to Hospitals and Cleaning Columns

- We clean the `district` column in `hospital_df` by converting to lowercase and stripping whitespace.  
- Using `district_lookup`, we map each hospital to its corresponding `district_id`.  
- Extra columns (`hosp_disp_code`, `taluka`, original `district`, and temporary `district_clean`) are removed to match the **database schema**.


In [ ]:
hospital_df["district_clean"] = hospital_df["district"].str.lower().str.strip()
hospital_df["district_id"] = hospital_df["district_clean"].map(district_lookup)
hospital_df.drop(columns=["hosp_disp_code", "taluka", "district", "district_clean"], inplace=True)
hospital_df.head()

In [74]:
hospital_df.to_csv("../data/maharashtra/hospital.csv")

### 📍 Adding Latitude and Longitude to Hospital Data

In this step, we enrich the `hospital.csv` dataset by adding **geographic coordinates** (latitude and longitude) for each hospital using two levels of location data:

1. **Pincode-level coordinates:**  
   - We match each hospital’s `pincode` with the corresponding `Latitude` and `Longitude` from the `pincode.csv` file.  
   - This provides accurate coordinates for most hospitals.

2. **District-level fallback:**  
   - For hospitals where pincode coordinates are missing, we use the corresponding **district’s latitude and longitude** from `district.csv` as a fallback.  
   - This ensures that every hospital has valid coordinates for mapping and spatial analysis.

After this step, all hospitals have complete latitude and longitude values, making the dataset ready for geospatial visualization and analysis.


In [109]:
hospital_df = pd.read_csv("../data/maharashtra/hospital.csv", index_col=0)
pincode_df = pd.read_csv("../data/pincode.csv", dtype={"Pincode": "string"}, low_memory=False)
pincode_df["Latitude"] = pd.to_numeric(pincode_df["Latitude"], errors="coerce")
pincode_df["Longitude"] = pd.to_numeric(pincode_df["Longitude"], errors="coerce")

## Adding Latitude and Longitude to Hospitals Using Pincode

This step enriches the `Hospital` table with geographic coordinates:

- We clean the `pincode` in `hospital_df` to remove decimals and whitespace.
- Using `pincode.csv`, we extract unique `Pincode` entries with their `Latitude` and `Longitude`.
- We merge these coordinates into `hospital_df` based on the cleaned pincode.
- Columns are renamed to `latitude` and `longitude` to match the database schema, and temporary merge columns are removed.

This allows each hospital to have accurate geographic coordinates for mapping and spatial analysis.


In [111]:
hospital_df['pincode_clean'] = hospital_df['pincode'].astype(str).str.split('.').str[0].str.strip()
pincode_unique = pincode_df[['Pincode', 'Latitude', 'Longitude']].drop_duplicates(subset='Pincode')

hospital_df = (
    hospital_df
    .merge(
        pincode_unique,
        left_on='pincode_clean',
        right_on='Pincode',
        how='left'
    )
    .set_index(hospital_df.index)
)

hospital_df.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'}, inplace=True)
hospital_df.drop(columns=['pincode_clean', 'Pincode'], inplace=True)

In [112]:
hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2370 entries, 1 to 2367
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hospital_name        2370 non-null   object 
 1   address              2096 non-null   object 
 2   pincode              2095 non-null   float64
 3   mco_contact_number   2088 non-null   float64
 4   total_beds           2370 non-null   int64  
 5   hospital_type        2365 non-null   object 
 6   government_sub_type  75 non-null     object 
 7   district_id          2370 non-null   int64  
 8   latitude             1917 non-null   float64
 9   longitude            1912 non-null   float64
dtypes: float64(4), int64(2), object(4)
memory usage: 203.7+ KB


### Observation: Missing Latitude and Longitude in Hospital Data

After merging the hospital dataset with the `pincode.csv` data, the `hospital_df` still contains some null values in the `latitude` and `longitude` columns:

- Total hospitals: 2370  
- Hospitals with non-null latitude: 1917  
- Hospitals with non-null longitude: 1912  

Some hospitals do not have latitude and longitude because their pincodes either do not exist in the reference dataset or are missing/incorrect in the hospital data. These null values indicate that geocoding by pincode alone is not sufficient for all entries.


### Filling Missing Hospital Coordinates with District Coordinates

For hospitals that still have null latitude or longitude after merging with `pincode.csv`, the corresponding district coordinates from `district.csv` are used as a fallback.  

- Merge hospital data with `district_df` on `district_id` to bring district latitude and longitude.  
- Fill missing hospital latitude and longitude with the district values.  
- Drop the temporary district coordinate columns after filling.

**Result:**  
All hospitals now have latitude and longitude, either from their pincode or from their district.


In [113]:
hospital_df = hospital_df.merge(
    district_df[['latitude', 'longitude']],
    left_on='district_id',
    right_index=True,
    suffixes=('', '_district')
)

hospital_df['latitude'] = hospital_df['latitude'].fillna(hospital_df['latitude_district'])
hospital_df['longitude'] = hospital_df['longitude'].fillna(hospital_df['longitude_district'])

hospital_df = hospital_df.drop(columns=['latitude_district', 'longitude_district'])

In [114]:
hospital_df = hospital_df.rename(columns={"government_sub_type": "government_subtype"})

hospital_df = hospital_df[
    [
        "district_id",
        "hospital_name",
        "address",
        "pincode",
        "latitude",
        "longitude",
        "mco_contact_number",
        "total_beds",
        "hospital_type",
        "government_subtype"
    ]
]

hospital_df.head()

,district_id,hospital_name,address,pincode,latitude,longitude,mco_contact_number,total_beds,hospital_type,government_subtype
hospital_id,,,,,,,,,,
1,35,INDRAVATI HOSPITAL & RESEARCH CENTRE,"Plot no. GHANSOLI VILLAGE, RCC HOUSE, ARJUNE W...",400701.0,19.124389,72.997611,8.452863e+09,30,Corporate,NaN
2,35,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,"C7, Budhyadev Mandir Marg, Sector 4, Sanpada, ...",400705.0,19.071861,73.007444,9.702694e+09,100,Corporate,NaN
3,17,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,NaN,NaN,19.446041,72.363737,NaN,15,Corporate,NaN
4,19,SANJEEVANI HOSPITAL,"NEAR TAHSIL OFFICE, GOTMARE LAYOUT, BRAHMI KA...",441501.0,21.231603,78.917390,9.860448e+09,30,Corporate,NaN
5,35,A & G HOSPITAL,Suchak House murbad road opp karnik road kalya...,421301.0,19.535300,76.144900,9.326115e+09,50,Corporate,NaN


In [116]:
hospital_df.to_csv("../data/maharashtra/hospital.csv")

### 🧭 Improving Hospital Location Accuracy with Targeted Geocoding

To ensure accurate visualization of hospitals on the map, we enhance the dataset with **precise latitude and longitude values** using the **OpenStreetMap Nominatim API**.

Previously, hospital coordinates were derived from **pincode-level data**, which caused multiple hospitals within the same pincode to share identical coordinates.  
To fix this, we now perform **targeted geocoding** — only for hospitals that share duplicate latitude and longitude values.

Our approach:
1. Identify hospitals with duplicate coordinates.  
2. Use **Nominatim** to geocode only those hospitals, based on their name, address, and district.  
3. If a geocode request succeeds, update the hospital’s latitude and longitude.  
4. If geocoding fails, retain the existing coordinates.  
5. Apply a small **random jitter (±0.001°)** to remaining duplicates to prevent visual overlap.

This ensures:
- Efficient use of the geocoding API.  
- Improved spatial precision.  
- Reduced overlap while keeping stable data for non-duplicate entries.


In [44]:
district_df = pd.read_csv("../data/maharashtra/district.csv")
hospital_df = pd.read_csv("../data/maharashtra/hospital.csv")

### 🔍 Identify Hospitals with Duplicate Coordinates

We first find all hospitals that share the same latitude and longitude.
These hospitals are candidates for re-geocoding.


In [45]:
# Identify hospitals with duplicate lat/lon pairs
duplicate_coords = hospital_df[hospital_df.duplicated(subset=["latitude", "longitude"], keep=False)]
print(f"Total hospitals with duplicate coordinates: {len(duplicate_coords)}")
duplicate_coords.head()

Total hospitals with duplicate coordinates: 2100


,hospital_id,district_id,hospital_name,address,pincode,latitude,longitude,mco_contact_number,total_beds,hospital_type,government_subtype
0,1,35,INDRAVATI HOSPITAL & RESEARCH CENTRE,"Plot no. GHANSOLI VILLAGE, RCC HOUSE, ARJUNE W...",400701.0,19.124389,72.997611,8.452863e+09,30,Corporate,NaN
1,2,35,MAHATMA PHULE CHARITABLE TRUST HOSPITAL,"C7, Budhyadev Mandir Marg, Sector 4, Sanpada, ...",400705.0,19.071861,73.007444,9.702694e+09,100,Corporate,NaN
2,3,17,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,NaN,NaN,19.446041,72.363737,NaN,15,Corporate,NaN
3,4,19,SANJEEVANI HOSPITAL,"NEAR TAHSIL OFFICE, GOTMARE LAYOUT, BRAHMI KA...",441501.0,21.231603,78.917390,9.860448e+09,30,Corporate,NaN
4,5,35,A & G HOSPITAL,Suchak House murbad road opp karnik road kalya...,421301.0,19.535300,76.144900,9.326115e+09,50,Corporate,NaN


### 🌐 Define the Geocoding Function

We use the **Nominatim API** from OpenStreetMap to fetch accurate coordinates based on hospital name, address, and district.


In [68]:
def geocode_location(name, district):
    """Fetch latitude and longitude from Nominatim API."""
    try:
        query = f"{name}, {district} District, Maharashtra, India"
        url = f"https://nominatim.openstreetmap.org/search"
        params = {"q": query, "format": "json", "limit": 1}
        headers = {"User-Agent": "hospital-geocoder-v1"}
        
        response = requests.get(url, params=params, headers=headers, timeout=10)
        data = response.json()
        
        if data:
            return float(data[0]["lat"]), float(data[0]["lon"])
    except Exception:
        return None
    return None

In [74]:
def clean_text(text):
    if pd.isna(text):
        return ""
    return str(text).replace("%E2%80%93", "-").replace("&", "and")

test_row = hospital_df.loc[22]
district_name = district_df.loc[
    district_df["district_id"] == test_row["district_id"], "district_name"
].values[0]

query = f"{clean_text(test_row['hospital_name']).capitalize()}, {district_name.capitalize()} District, Maharashtra, India"
print("Query:", query)

result = geocode_location(
    clean_text(test_row["hospital_name"]).capitalize(),
    district_name.capitalize()
)

print("Geocode result:", result)


Query: Adhar superspeciality hospital, Pune District, Maharashtra, India
Geocode result: None


### 🚫 Why We Didn’t Use Nominatim Geocoding

Although geocoding seemed ideal for improving hospital location accuracy, we found that most hospitals could not be resolved by **OpenStreetMap Nominatim** due to:
- Missing or non-standardized addresses in the dataset.  
- Hospitals not listed in OpenStreetMap’s database.  
- Frequent `None` responses even for valid names and districts.  

Because of these limitations, using Nominatim for large-scale geocoding proved **infeasible** and **unreliable**.  

Hence, we decided to:
1. **Retain existing pincode-based coordinates**, which at least locate hospitals approximately within the right region.  
2. **Apply a small random jitter** to hospitals sharing the same coordinates, ensuring they appear separately on the map for better visualization.


### 🔄 Apply Jitter to Duplicate Coordinates

To visually separate overlapping hospitals on the map, we apply a small random offset (±0.001°) to latitude and longitude values for all duplicate entries.  
This jitter ensures that points no longer perfectly overlap, improving readability on interactive visualizations.


In [75]:
def apply_jitter(df):
    jitter_strength = 0.001
    for idx in df.index:
        df.at[idx, "latitude"] += np.random.uniform(-jitter_strength, jitter_strength)
        df.at[idx, "longitude"] += np.random.uniform(-jitter_strength, jitter_strength)
    return df

# Apply jitter to duplicates in hospital_df
hospital_df.loc[duplicate_coords.index] = apply_jitter(hospital_df.loc[duplicate_coords.index])

### ✅ Verify Coordinate Uniqueness After Jitter

We now verify that hospitals no longer share identical coordinates after applying the jitter.


In [76]:
# Re-check for duplicate coordinates
remaining_duplicates = hospital_df[hospital_df.duplicated(subset=["latitude", "longitude"], keep=False)]
print(f"Remaining hospitals with duplicate coordinates: {len(remaining_duplicates)}")

Remaining hospitals with duplicate coordinates: 0


In [78]:
hospital_df.to_csv("../data/maharashtra/hospital.csv", index=False)

## 🧭 Problem: Mumbai District Coordinates Are Incorrect

While plotting hospitals on the map, we noticed that **many hospitals from Mumbai appear in the sea**, off the western coast of India.  
This indicates that the **latitude and longitude values for the “MUMBAI” district** (and possibly others nearby) are incorrect in our dataset.

We’ll inspect these coordinates in `district_df`, confirm the issue, and then correct them appropriately.


In [79]:
district_df = pd.read_csv("../data/maharashtra/district.csv")
hospital_df = pd.read_csv("../data/maharashtra/hospital.csv")

## 🔍 Check Coordinates of Mumbai District

Let’s inspect the existing latitude and longitude values for the “MUMBAI” district.     
If these coordinates are far west of mainland India (for example near 72°E, 19°N), then they are likely located **in the Arabian Sea**.


In [81]:
# Inspect current coordinates for Mumbai-related districts
district_df[
    district_df["district_name"].str.upper().str.contains("MUMBAI")
][["district_id", "district_name", "latitude", "longitude"]]

,district_id,district_name,latitude,longitude
16,17,MUMBAI,19.446041,72.363737
17,18,MUMBAI SUBURBAN,19.118783,72.873937


## 🌊 Observation: Mumbai District Lies in the Sea

The output shows that the current coordinates for **MUMBAI** (and possibly **MUMBAI SUBURBAN**) are located near:
> Latitude = 19.446041 
> Longitude = 72.363737  

However, these longitudes (~72.3°E) are **too far west**, placing the point in the **Arabian Sea**.  
Mumbai city’s approximate coordinates should be around **(18.9582°N, 72.8321°E)** — inland, along the western coast.

We’ll fix these district coordinates next.


## 🧩 Step 2: Correct Mumbai District Coordinates

Only the **MUMBAI** district had incorrect coordinates — it was located offshore in the Arabian Sea.  
We’ll fix it using a verified centroid near South Mumbai.

| District | Correct Latitude | Correct Longitude |
|-----------|-----------------|------------------|
| MUMBAI | 18.9582 | 72.8321 |

These coordinates accurately place the district inland near Fort / Colaba region.


In [82]:
# Correct coordinates for Mumbai only
corrections = {
    "MUMBAI": (18.9582, 72.8321),
}

for name, (lat, lon) in corrections.items():
    mask = district_df["district_name"].str.upper() == name
    district_df.loc[mask, ["latitude", "longitude"]] = (lat, lon)

district_df[
    district_df["district_name"].str.upper().str.contains("MUMBAI")
][["district_name", "latitude", "longitude"]]

,district_name,latitude,longitude
16,MUMBAI,18.958200,72.832100
17,MUMBAI SUBURBAN,19.118783,72.873937


## 🩺 Fix Mumbai Hospital Coordinates by Applying Offset

The old district coordinates for *Mumbai* were located in the sea.  
Instead of assigning all hospitals a single fixed location,  
we compute the **difference (offset)** between the old and corrected district coordinates and apply that shift to all hospitals under the Mumbai district.

This keeps the hospitals' relative positions intact while moving them inland.


In [ ]:
# Old and new coordinates for Mumbai district
old_lat, old_lon = 19.446041, 72.363737
new_lat, new_lon = 18.9582, 72.8321

# Calculate offsets
lat_diff = new_lat - old_lat
lon_diff = new_lon - old_lon

# Apply the offset to all hospitals in the Mumbai district
mumbai_id = district_df.loc[
    district_df["district_name"].str.upper() == "MUMBAI", "district_id"
].iloc[0]

mask = hospital_df["district_id"] == mumbai_id

hospital_df.loc[mask, "latitude"] += lat_diff
hospital_df.loc[mask, "longitude"] += lon_diff

print(f"✅ Shifted coordinates for {mask.sum()} hospitals in Mumbai district.")

✅ Shifted coordinates for 92 hospitals in Mumbai district.


,hospital_name,latitude,longitude
2,REHBAR FOUNDATION MALIK RAKIYA KHATOON DIALYSI...,18.957765,72.831658
19,ACHARYA SHRI RAMESH DIALYSIS CENTER,18.958266,72.831911
20,ACI CUMBALLA HILL HOSPITAL,18.957538,72.833055
39,ALL CURE SUPER SPECIALITY HOSPITAL,18.957479,72.832238
59,APEX HOSPITALS MULUND (UNIT OF NOBLE MEDICAL C...,18.958562,72.832532


In [84]:
district_df.to_csv("../data/maharashtra/district.csv", index=False)
hospital_df.to_csv("../data/maharashtra/hospital.csv", index=False)